# Functions

## log.py

In [64]:
# replace these everywhere
def msginfo(verbose, msg):
    msg2(verbose,msg)
    
def msg2(verbose, msg):
    if verbose >= 2:
        print(msg)

def msg1(verbose, msg):
    if verbose >= 1:
        print(msg)

## reference.py

In [32]:
import requests

def download_gene_symbols(output_filepath = "data/raw/gene_info.gz", url = "https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz", verbose=0):
    response = requests.get(url)
    response.raise_for_status()

    with open(output_filepath, "wb") as file:
        file.write(response.content)

    msginfo(verbose, "Download completed.")

def extract_gene_data(filepath = "data/raw/gene_info.gz"):
    """ returns 3 sets """
    import gzip

    gene_symbols = set()
    dbxrefs = set()
    gene_synonyms = set()

    with gzip.open(filepath, "rt") as file:
        for line in file:
            if not line.startswith("#"):
                fields = line.strip().split("\t")
                gene_symbols.add(fields[2])

                # Extract dbXrefs
                dbxref_field = fields[5]
                if dbxref_field != "-":
                    identifiers = dbxref_field.split("|")
                    dbxrefs.update(identifiers)

                # Extract gene synonyms
                synonym_field = fields[4]
                if synonym_field != "-":
                    synonyms = synonym_field.split("|")
                    gene_synonyms.update(synonyms)

    return gene_symbols, dbxrefs, gene_synonyms

## search_terms.py

In [3]:
def read_search_stop(search_file, stop_words):
    # Load search terms from the file into a set
    search_terms=[]
    with open(search_file, 'r') as f:
        search_terms = [line.strip() for line in f]
    return search_terms

def filter_search_terms(search_terms, stop_words):
    filtered_terms = []
    matched_stop_words = []
    
    # get the matched stop words and keep in original case
    for term in search_terms:
        if term.lower() in stop_words:
            matched_stop_words.append(term)

    # get the filtered terms, lower case them
    filtered_terms = [term.lower() for term in search_terms if term.lower() not in stop_words]

    # add the two lists together so the matched terms retain original case
    # this will allow for finding genes that are also common english words
    final_terms = filtered_terms + matched_stop_words
    return (final_terms, filtered_terms, matched_stop_words)


## stop_words.py

In [5]:
from typing import List, Dict, Set

import nltk
from nltk import FreqDist
import string

def fetch_brown_corpus():
    from nltk.corpus import brown
    corpus = 'brown'
    # Select a specific category from the Brown Corpus for analysis
    category = 'learned'
    # Load the Brown Corpus
    nltk.download('brown')
    # Get the words from the selected category
    words = brown.words(categories=category)
    return words

def read_search_stop(search_file, stop_words):
    # Load search terms from the file into a set
    search_terms=[]
    with open(search_file, 'r') as f:
        search_terms = [line.strip() for line in f]
    return search_terms

def filter_search_terms(search_terms, stop_words):
    filtered_terms = []
    matched_stop_words = []
    
    # get the matched stop words and keep in original case
    for term in search_terms:
        if term.lower() in stop_words:
            matched_stop_words.append(term)

    # get the filtered terms, lower case them
    filtered_terms = [term.lower() for term in search_terms if term.lower() not in stop_words]

    # add the two lists together so the matched terms retain original case
    # this will allow for finding genes that are also common english words
    final_terms = filtered_terms + matched_stop_words
    return (final_terms, filtered_terms, matched_stop_words)

def create_stop_words(frequency_list_outpath, custom_words) -> Set:
    from nltk.corpus import stopwords

    # Download the stopwords corpus
    nltk.download('stopwords')

    # Load the existing stop words
    stop_words = set(stopwords.words('english'))

    # Read the words from the frequency_list.txt file
    with open(frequency_list_outpath, 'r') as file:
        frequency_words = file.read().splitlines()

    # Add the frequency words to the stop words set
    stop_words.update(frequency_words)

    # Add custom words
    stop_words.update(custom_words)
    return stop_words


## fetch.py

In [93]:
import os
import subprocess
import urllib.request
from urllib.error import URLError
from shutil import rmtree

def check_disk_space(predicted_size, download_dir, verbose):
    required_space = predicted_size
    available_space = os.statvfs(download_dir).f_frsize * os.statvfs(download_dir).f_bavail
    required_space_human = subprocess.check_output(['numfmt', '--to=iec-i', '--suffix=B', str(required_space)]).decode().strip()
    available_space_human = subprocess.check_output(['numfmt', '--to=iec-i', '--suffix=B', str(available_space)]).decode().strip()

    msg1(verbose, f"Predicted download size = {required_space_human}, Available space = {available_space_human}")

    if required_space > available_space:
        print(f"Insufficient disk space! Required: {required_space_human}, Available: {available_space_human}")
        exit(1)

def download_file(url, file_path, verbose):
    try:
        urllib.request.urlretrieve(url, file_path)
    except URLError as e:
        msg1(verbose, f"Error downloading file: {url}")
        msg1(verbose, f"Reason: {str(e.reason)}")
        #exit(1)

def verify_md5(file_path, md5_file_path, verbose):
    try:
        output = subprocess.check_output(['md5sum', '-c', os.path.basename(md5_file_path)], cwd=os.path.dirname(md5_file_path), stderr=subprocess.DEVNULL).decode()
        #output = subprocess.check_output(['md5sum', '-c', md5_file_path], stderr=subprocess.DEVNULL).decode()
        if "OK" in output:
            msg2(verbose, f"{md5_file_path}: OK - MD5 checksum verification succeeded.")
        else:
            msg1(verbose, f"ERROR: {md5_file_path}: FAILED - MD5 checksum verification failed.")
    except subprocess.CalledProcessError:
        msg1(verbose, f"ERROR: {md5_file_path}: FAILED - MD5 checksum verification failed.")


## parse.py

In [6]:
import pandas as pd
import numpy as np
import re

def extract_data(element):
    data = {}
    data['PMID'] = element.findtext('MedlineCitation/PMID')
    data['Title'] = element.findtext('MedlineCitation/Article/ArticleTitle')
    data['Abstract'] = element.findtext('MedlineCitation/Article/Abstract/AbstractText')
    data['Journal'] = element.findtext('MedlineCitation/Article/Journal/Title')
    data['PublicationDate'] = element.findtext('MedlineCitation/Article/Journal/JournalIssue/PubDate/Year')
    data['JournalTitle'] = element.findtext('MedlineCitation/Article/Journal/Title')
    data['ArticleType'] = element.findtext('MedlineCitation/Article/PublicationTypeList/PublicationType')
    
    # Extract the descriptor names and qualifier names from the XML
    mesh_headings = element.findall('.//MeshHeading')
    mesh_heading_list = []
    for heading in mesh_headings:
        descriptor_name = heading.findtext('DescriptorName')
        qualifier_names = [qualifier.text for qualifier in heading.findall('QualifierName')]
        mesh_heading_list.append(descriptor_name)
        mesh_heading_list.extend(qualifier_names)
    data['MeshHeadingList'] = ','.join(mesh_heading_list)
                                        
    publication_types = element.findall('MedlineCitation/Article/PublicationTypeList/PublicationType')
    data['PublicationTypeList'] = ",".join([ptype.text for ptype in publication_types])
    
    return data

def prune_df(df, length_threshold = 405, verbose=2):
    # exclude articles with no abstract, no date, or abstracts that are too short (less than length_threshold letters)
    pruned_df = df[df['Abstract'].notna() & df['PublicationDate'].notna()]

    # cut out any short articles
    all_pruned = len(pruned_df)
    msg2(verbose, f"Number of all abstracts before pruning short articles = {all_pruned}")
    pruned_df = pruned_df[pruned_df['Abstract'].str.len() >= length_threshold]
    long_pruned = len(pruned_df)
    msg2(verbose, f"Number after pruning short articles = {long_pruned}")
    msg2(verbose, f"Number discarded for being too short: {all_pruned - long_pruned}")

    return pruned_df

def get_pub_df(filename, inpath, outpath, length_threshold, prune=True,verbose=0):
    import gzip
    import xml.etree.ElementTree as ET
    import pandas as pd

    pubmed_filepath = os.path.join(inpath, filename)
    # Open the gzip'd XML file
    with gzip.open(pubmed_filepath, 'rb') as f:
        # Read the contents of the gzip'd file
        gzip_content = f.read()

    # Parse the XML content using ElementTree
    root = ET.fromstring(gzip_content)

    # Extract data from each article and store in a list
    articles = []
    for article in root.findall('.//PubmedArticle'):
        articles.append(extract_data(article))

    # Create a DataFrame from the list of articles
    df = pd.DataFrame(articles)
    df = df.drop_duplicates()
    if prune:
        msg2(verbose, f"Number of all articles:{len(df)}")
        df = prune_df(df, length_threshold = length_threshold, verbose=verbose)
        msg2(verbose, f"Number of pruned articles:{len(df)}")
    
    # convert objects to simple types
    df['PublicationDate'] = df['PublicationDate'].astype(int)

    return df


## models.py

In [82]:
import os
from typing import List, Dict, Set
from pydantic import BaseModel, root_validator

class ReferenceData(BaseModel):
    """
    Files for retrieving and transforming reference gene information
    """
    
    def __init__(self, *args, **kwargs):
        super(ReferenceData, self).__init__(*args, **kwargs)

        self.version_root = os.path.join(os.getcwd(), str(self.version), self.data_root)
        msg2(self.verbose, f"version_root={self.version_root}")
        
        # make directory structure
        os.makedirs(self.version_root, exist_ok=True)
        os.makedirs(self.raw_path(), exist_ok=True)
        os.makedirs(self.reference_path(), exist_ok=True)
        os.makedirs(self.dbxref_path(), exist_ok=True)
        os.makedirs(self.search_path(), exist_ok=True)
        os.makedirs(self.pub_inpath(), exist_ok=True)
        os.makedirs(self.pub_outpath(), exist_ok=True)
        os.makedirs(self.genes_outpath(), exist_ok=True)
        
        msg2(self.verbose, "Created directory structure.")

    ncbi_gene_info_url: str = "https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz"
    
    data_root: str  = "data/"
    """ full path to where all the raw, reference, and generated data are stored """
    
    raw_data_path: str = "raw/"
    
    reference_data_path: str = "reference/"
    
    dbxref_reference_data_path: str = "dbxrefs/"
    
    dbxrefs: List = [] 
    """ if empty, it will just get filled with a list of all the files created in the m.dbxref_path() """
        
    gene_info_filename: str = "gene_info.gz"
    
    gene_symbols_filename: str = "gene_symbols.txt"
    
    gene_synonyms_filename: str = "gene_synonyms.txt"
    
    search_terms_path: str = "search_terms/"
    
    frequency_list_filename: str = "frequency_list.txt"
    
    corpus_stop_word_list_length:int = 4000
    
    # old lsit
    #custom_stop_words:List = ['ago', 'aim', 'amid', 'april', 'arch', 'bed', 'bite', 'bug', 'co', 'crop', 'damage', 'et', 
    #                'fast', 'fat', 'fate', 'gap', 'genesis', 'ii', 'iv', 'lamp', 'laser', 'mater', 'melt', 'mice', 'minor', 'mv', 'net', 
    #                'not', 'race', 'rank', 'se', 'sink', 'soft', 'spatial', 'steel', 'stop', 'tau', 'traits', 'via']
    
    # may overlap somewhat with stop words
    custom_stop_words:List = ['ago', 'aim', 'amid', 'april', 'arch', 'bed', 'bite', 'bug', 'cage', 'co', 'crop',
                    'damage', 'danger', 'digit', 'et', 'fast', 'fat', 'fate', 'fire', 'flower', 'gap', 'genesis', 'gov', 'gpa', 'grasp',
                    'ii', 'inos', 'iv', 'killer', 'lab', 'lamp', 'laser', 'map', 'mask', 'mater', 'melt', 'mice', 'minor', 'miss', 'mv',
                    'nail', 'net', 'not', 'osf', 'pan', 'par', 'pha', 'rab', 'race', 'rain', 'rank',
                    'san', 'sand', 'se', 'sink', 'soft', 'spatial', 'spin', 'spp', 'steel', 'stop',
                    'storm', 'tactile', 'tau', 'theta', 'tip', 'traits', 'via']

    search_terms_filename: str = "search_terms.txt"
    
    filtered_terms_filename: str = "filtered_terms.txt"
    
    abstract_inpath: str = "pubs/"
    
    refresh_abstract_xml_files: bool = False
    """ Set to True to overwrite downloaded NCBI XML abstract files and checksum files """
    
    num_abstract_xml_files: int
    """ Number of NCBI XML files to download; Set this to -1 to get all abstracts """
    
    abstract_outpath: str = "csvpubs/"

    abstract_length_threshold: int = 405

    abstract_genes_outpath: str = "genes/"

    
    #path functions
    def raw_path(self):
        return os.path.join(os.getcwd(), self.version_root, self.raw_data_path)
    def reference_path(self):
        return os.path.join(os.getcwd(), self.version_root, self.reference_data_path)
    def dbxref_path(self):
        return os.path.join(os.getcwd(), self.version_root, self.reference_data_path, self.dbxref_reference_data_path)
    def search_path(self):
        return os.path.join(os.getcwd(), self.version_root, self.search_terms_path)
    def pub_inpath(self):
        return os.path.join(os.getcwd(), self.version_root, self.raw_data_path, self.abstract_inpath)
    def pub_outpath(self):
        return os.path.join(os.getcwd(), self.version_root, self.abstract_outpath)
    def genes_outpath(self):
        return os.path.join(self.pub_outpath(), self.abstract_genes_outpath)

    verbose: int = 0
    """ 0 prints nothing, 1 prints errors and warnings, 2 prints info """
    
    debug: bool = False
    """ Prints very detailed debuggin messages """
    
    version: str = None
    """ If None, version will e set to timestamp """
    
    rand_seed: int = 42
    """ if None, reproducible within the same release"""
    
    version_root: str = None
    """ leave this alone it will be computed as data_root/version """
    
    
    @root_validator(pre=False, skip_on_failure=True)
    def valid_paths_for_data_tree(cls, v: Dict) -> Dict:
        data_root = v.get("data_root")
        version = v.get("version")
        
        import os
        
        if version is None:
            import subprocess
            
            # use miliseconds and create new directory structure
            version = str(subprocess.check_output(["date", "+%1N.%S_%M_%H_%Y_%m_%d"]).strip().decode())
            v["version"] = version
        
        if data_root is None:
            raise Exception("Error: data_root cannot be None.")
            
        version_root = os.path.join(os.getcwd(), data_root, str(version))
        v["version_root"] = version_root
        
        if os.path.exists(version_root):
            msg2(f"Using existing data director {version_root}")
            
        return v
    


## api.py

In [95]:
def create_gene_reference_data(m: ReferenceData):
    
    raw_gene_info_filepath = os.path.join(m.raw_path(), m.gene_info_filename)
    reference_gene_symbols_filepath = os.path.join(m.reference_path(), m.gene_symbols_filename)
    reference_gene_synonyms_filepath = os.path.join(m.reference_path(), m.gene_synonyms_filename)
    dbxref_path = m.dbxref_path()
    verbose = m.verbose
    url = m.ncbi_gene_info_url
    
    # Download the gene symbols file
    download_gene_symbols(output_filepath = raw_gene_info_filepath, url = url, verbose = verbose)

    # Extract gene data
    gene_symbols, dbxrefs, gene_synonyms = extract_gene_data(filepath = raw_gene_info_filepath)

    # Save gene symbols to a file
    with open(reference_gene_symbols_filepath, "w") as file:
        for symbol in gene_symbols:
            file.write(symbol + "\n")

    msg2(verbose, f"Gene symbols saved to {reference_gene_symbols_filepath}")

    # Save dbXrefs to separate files
    for identifier in dbxrefs:
            identifier_parts = identifier.split(":")
            identifier_type = identifier_parts[0].replace('/','_')
            identifier_value = ":".join(identifier_parts[1:])
            filename = f"{dbxref_path}/{identifier_type}.txt"
            with open(filename, "a") as file:
                file.write(identifier_value + "\n")

    """
    for identifiers in dbxrefs:
        for identifier in identifiers:
            identifier_parts = identifier.split(":")
            identifier_type = identifier_parts[0].replace('/','_')
            identifier_value = ":".join(identifier_parts[1:])
            filename = f"{dbxref_path}/{identifier_type}.txt"
            with open(filename, "a") as file:
                file.write(identifier_value + "\n")
    """
    msg2(verbose, "dbXrefs saved to individual files.")

    # Save gene synonyms to a file
    with open(reference_gene_synonyms_filepath, "w") as file:
        for synonym in gene_synonyms:
            file.write(synonym + "\n")

    msg2(verbose, f"Gene synonyms saved to {reference_gene_synonyms_filepath}")

def create_frequency_list(m: ReferenceData) -> List:
    
    frequency_list_outpath = os.path.join(m.search_path(), m.frequency_list_filename)
    stop_word_list_length = m.corpus_stop_word_list_length
    verbose = m.verbose
    
    import nltk
    from nltk import FreqDist
    import string

    words = fetch_brown_corpus()

    # Remove punctuation and convert to lowercase
    words = [word.lower() for word in words if word not in string.punctuation and word.isalnum()]

    # Compute the frequency distribution of words
    freq_dist = FreqDist(words)

    # Get the most frequent words
    most_common_words = freq_dist.most_common(stop_word_list_length)

    # Write the frequency list to a file
    with open(frequency_list_outpath, 'w') as file:
        for word, frequency in most_common_words:
            file.write(word + '\n')
    msg2(verbose, f"Wrote {frequency_list_outpath}")
    return most_common_words

def create_search_terms_file(m: ReferenceData):
    import os

    dbxrefs = m.dbxrefs
    dbxrefs_path = m.dbxref_path()  
    gene_symbols_filepath = os.path.join(m.reference_path(), m.gene_symbols_filename)
    gene_synonyms_filepath = os.path.join(m.reference_path(), m.gene_synonyms_filename)
    search_terms_filepath = os.path.join(m.search_path(), m.search_terms_filename)
    verbose = m.verbose

    if dbxrefs == []:
        # Get a list of all files in the directory
        dbxrefs = os.listdir(dbxrefs_path)
        # Filter out directories from the list
        dbxrefs = [f for f in dbxrefs if os.path.isfile(os.path.join(dbxrefs_path, f))]
  
    
    with open(f"{search_terms_filepath}.unsorted", "w") as outfile:
        for ref in dbxrefs:
            with open(os.path.join(dbxrefs_path, ref)) as infile:
                sorted_lines = sorted(set(infile.readlines()))
                outfile.writelines(sorted_lines)
                #outfile.write(infile.read())

        with open(gene_symbols_filepath) as infile:
            sorted_lines = sorted(set(infile.readlines()))
            outfile.writelines(sorted_lines)
            #outfile.write(infile.read())

        with open(gene_synonyms_filepath) as infile:
            sorted_lines = sorted(set(infile.readlines()))
            outfile.writelines(sorted_lines)
            #outfile.write(infile.read())

    # Sort and remove duplicates from the search terms file
    search_terms_unsorted_filepath = f"{search_terms_filepath}.unsorted"
    os.system(f"sort -u {search_terms_unsorted_filepath} | grep -v not > {search_terms_filepath}")

    msg2(verbose, f"Created {search_terms_filepath}.")
    msg2(verbose, f"Created {search_terms_unsorted_filepath} - can be removed.")
    line_count = sum(1 for line in open(search_terms_filepath))
    msg2(verbose, f"Number of lines in {search_terms_filepath}: {line_count}")

def create_filtered_search_terms(m: ReferenceData) -> List:
    
    search_file = os.path.join(m.search_path(), m.search_terms_filename)
    frequency_list_outpath = os.path.join(m.search_path(), m.frequency_list_filename)
    custom_words = m.custom_stop_words
    final_file = os.path.join(m.search_path(), m.filtered_terms_filename)
    verbose = m.verbose
    
    stop_words = create_stop_words(frequency_list_outpath, custom_words)
    
    search_terms = read_search_stop(search_file = search_file, stop_words = stop_words)
    msg2(verbose, f"Number of original search_terms:{len(search_terms)}")
    final_terms, filtered_terms, matched_stop_words = filter_search_terms(search_terms, stop_words)
    msg2(verbose, f"number of filtered_terms:{len(filtered_terms)}\nfinal number of final_terms:{len(final_terms)}\n number of matched_stop_words:{len(matched_stop_words)}\nmatched_stop_words={matched_stop_words}")
    if final_file is not None:
        with open(final_file, "w") as f:
            f.writelines('\n'.join(final_terms))
    msg2(verbose, f"Created {final_file}")
    return final_terms


def fetch_abstracts(m: ReferenceData):
    
    num_files = m.num_abstract_xml_files
    refresh = m.refresh_abstract_xml_files
    download_dir = m.pub_inpath()
    verbose = m.verbose
    
    """ This can probably be done faster with download_files.sh """ 
    msg2(verbose, f"Download Directory: {download_dir}")
    msg2(verbose, f"Number of abstracts to ensure have been downloaded: {num_files}")
    msg2(verbose, f"Refresh: {refresh}")

    # FTP settings
    ftp_host = "ftp.ncbi.nlm.nih.gov"
    ftp_path = "/pubmed/baseline/"

    # Retrieve file names and find the largest number
    #file_list = subprocess.check_output(['curl', '-s', f"ftp://{ftp_host}{ftp_path}"]).decode().splitlines()
    
    output = subprocess.check_output(['curl', '-s', f"ftp://{ftp_host}{ftp_path}"]).decode()
    file_list = [line.split()[-1] for line in output.splitlines() if line.endswith(".xml.gz")]

    msg2(verbose, f"Total number of NCBI abstract XML files: {len(file_list)}")
    latest_files = [file_name for file_name in file_list if file_name.startswith("pubmed23n") and file_name.endswith(".xml.gz")]
    latest_files.sort(reverse=True)
    latest_files = latest_files[:num_files]
    msg2(verbose, f"latest_files {num_files}: {latest_files}")

    # Check if enough files are available
    if len(latest_files) == 0:
        msg1(verbose, "Error: Insufficient number of files available!")
        exit(1)

    # Calculate total predicted size
    total_size = 0
    for file_name in latest_files:
        response = subprocess.check_output(['curl', '-sI', f"ftp://{ftp_host}{ftp_path}{file_name}"]).decode()
        file_size = int(response.split("Content-Length: ")[1].split("\r")[0])
        total_size += file_size

    # Check disk space before downloading
    check_disk_space(total_size, download_dir, verbose=verbose)

    # Download and check files
    for file_name in latest_files:
        md5_file_name = f"{file_name}.md5"
        file_path = os.path.join(download_dir, file_name)
        md5_file_path = os.path.join(download_dir, md5_file_name)

        # Refresh files that were previously downloaded?
        if not refresh:
            # No, so skip downloading those again

            # If one file or the other is missing, you still have to do a download
            # Here, just provide information as to which files are present.
            if os.path.isfile(file_path) and not os.path.isfile(md5_file_path):
                msg1(verbose, f"ERROR: Missing - {md5_file_path}; re-downloading now")
            if not os.path.isfile(file_path) and os.path.isfile(md5_file_path):
                msg1(verbose, f"ERROR: Missing - {file_path}; re-downloading now")

            if os.path.isfile(file_path) and os.path.isfile(md5_file_path):
                msg1(verbose, f"SKIP: {file_path} exists.")
                continue

        # Check file size
        response = subprocess.check_output(['curl', '-sI', f"ftp://{ftp_host}{ftp_path}{file_name}"]).decode()
        file_size = int(response.split("Content-Length: ")[1].split("\r")[0])

        msg2(verbose, f"File: {file_name}, Size: {file_size} bytes")

        # Download file
        msg2(verbose, f"WARNING: Downloading: {file_name} to {download_dir}")
        if os.path.isfile(file_path):
            os.remove(file_path)
        download_file(f"ftp://{ftp_host}{ftp_path}{file_name}", file_path, verbose)

        # Download MD5 file
        if os.path.isfile(md5_file_path):
            os.remove(md5_file_path)
        download_file(f"ftp://{ftp_host}{ftp_path}{md5_file_name}", md5_file_path, verbose)

        # Check MD5
        verify_md5(file_path, md5_file_path, verbose)

    total_size_human = subprocess.check_output(['numfmt', '--to=iec-i', '--suffix=B', str(total_size)]).decode().strip()
    msg2(verbose, f"Total size of abstract files: {total_size_human}")

def create_pubcsv_dataset(m: ReferenceData) -> List:
    """ Takes about 14min for 30 (2 per minute) """
    
    abstract_length_threshold = m.abstract_length_threshold
    pub_inpath = m.pub_inpath()
    pub_outpath = m.pub_outpath()
    verbose = m.verbose

    import os
    import glob
    
    csv_list = []
    # Iterate through files in the directory
    for filepath in glob.glob(os.path.join(pub_inpath, "pubmed*.xml.gz")):
        msg2(verbose, f"Converting file {filepath}")
        if os.path.isfile(filepath):
            filename = os.path.basename(filepath)
            df = get_pub_df(filename=filename, inpath=pub_inpath, outpath= pub_outpath, prune=True, length_threshold = abstract_length_threshold, verbose = verbose)
            csv_filepath = os.path.join(pub_outpath, f"{filename}.csv")
            df.to_csv(csv_filepath, header=False, index=False, sep="\t")
            msg2(verbose, f"Wrote file:{csv_filepath}")
            csv_list.append(csv_filepath)
            
    return(csv_list)

# Create data model

In [84]:
# exclue miRNA and MIM
m = ReferenceData(version = "v1", verbose = 2, 
                  num_abstract_xml_files = 5,
                  dbxrefs = ["AllianceGenome.txt", "Ensembl.txt", "HGNC.txt", "IMGT_GENE-DB.txt"]  )
m

version_root=/home/krobasky/prompt/end2end/v1/data/
Created directory structure.


ReferenceData(ncbi_gene_info_url='https://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz', data_root='data/', raw_data_path='raw/', reference_data_path='reference/', dbxref_reference_data_path='dbxrefs/', dbxrefs=['AllianceGenome.txt', 'Ensembl.txt', 'HGNC.txt', 'IMGT_GENE-DB.txt'], gene_info_filename='gene_info.gz', gene_symbols_filename='gene_symbols.txt', gene_synonyms_filename='gene_synonyms.txt', search_terms_path='search_terms/', frequency_list_filename='frequency_list.txt', corpus_stop_word_list_length=4000, custom_stop_words=['ago', 'aim', 'amid', 'april', 'arch', 'bed', 'bite', 'bug', 'cage', 'co', 'crop', 'damage', 'danger', 'digit', 'et', 'fast', 'fat', 'fate', 'fire', 'flower', 'gap', 'genesis', 'gov', 'gpa', 'grasp', 'ii', 'inos', 'iv', 'killer', 'lab', 'lamp', 'laser', 'map', 'mask', 'mater', 'melt', 'mice', 'minor', 'miss', 'mv', 'nail', 'net', 'not', 'osf', 'pan', 'par', 'pha', 'rab', 'race', 'rain', 'rank', 'san', 'sand', 'se', 'sink', 'sof

# Get human genes list

In [35]:
create_gene_reference_data(m)

Download completed.
Gene symbols saved to /home/krobasky/prompt/end2end/v1/data/reference/gene_symbols.txt
dbXrefs saved to individual files.
Gene synonyms saved to /home/krobasky/prompt/end2end/v1/data/reference/gene_synonyms.txt


# Create filtered_terms.txt - 3 API calls

In [36]:
_ = create_frequency_list(m)

[nltk_data] Downloading package brown to /home/krobasky/nltk_data...
[nltk_data]   Package brown is already up-to-date!


Wrote /home/krobasky/prompt/end2end/v1/data/search_terms/frequency_list.txt


In [50]:
create_search_terms_file(m)

Created /home/krobasky/prompt/end2end/v1/data/search_terms/search_terms.txt.
Created /home/krobasky/prompt/end2end/v1/data/search_terms/search_terms.txt.unsorted - can be removed.
Number of lines in /home/krobasky/prompt/end2end/v1/data/search_terms/search_terms.txt: 338143


In [51]:
final_terms = create_filtered_search_terms(m)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/krobasky/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Number of original search_terms:338143
number of filtered_terms:337951
final number of final_terms:338143
 number of matched_stop_words:192
matched_stop_words=['ABO', 'ACE', 'ACT', 'AF', 'AGO', 'AID', 'AIM', 'AIR', 'ALL', 'AM', 'AMID', 'AN', 'APRIL', 'APT', 'ARC', 'ARCH', 'ARM', 'ARMS', 'ART', 'AS', 'ASK', 'AT', 'BAD', 'BANK', 'BASE', 'BED', 'BEST', 'BITE', 'BOD', 'BORIS', 'BRIGHT', 'BUG', 'CAGE', 'CALL', 'CAN', 'CAR', 'CAT', 'CELL', 'CHIP', 'CO', 'CROP', 'DAMAGE', 'DANGER', 'DC', 'DIGIT', 'DO', 'END', 'ET', 'ETA', 'FACE', 'FACT', 'FAST', 'FAT', 'FATE', 'FIND', 'FIRE', 'FLOWER', 'FOR', 'GAP', 'GAS', 'Genesis', 'GET', 'GO', 'GOV', 'GPA', 'GRASP', 'GREAT', 'H', 'HAD', 'HAS', 'HE', 'hELD', 'HIS', 'hole', 'HOT', 'HR', 'iCE', 'ICE', 'IF', 'II', 'IMPACT', 'IN', 'INOS', 'IV', 'JET', 'KILLER', 'LAB', 'LAMP', 'LARGE', 'LASER', 'LED', 'LIGHT', 'LIME', 'LIMIT', 'MA', 'MAIL', 'MAP', 'MARCH', 'MARK', 'MARS', 'MASK', 'MASS', 'MATER', 'ME', 'MELT', 'MEN', 'Met', 'MET', 'MG', 'MICE', 'MINOR', 'MISS', 

In [52]:
len(final_terms)

338143

# Fetch ncbi article zips

In [113]:
# 60GB is needed to get all files
# 2 min/file ... ~ 2 days to get 'em all
m.num_abstract_xml_files=10
fetch_abstracts(m)

Download Directory: /home/krobasky/prompt/end2end/v1/data/raw/pubs/
Number of abstracts to ensure have been downloaded: 10
Refresh: False
Total number of NCBI abstract XML files: 1166
latest_files 10: ['pubmed23n1166.xml.gz', 'pubmed23n1165.xml.gz', 'pubmed23n1164.xml.gz', 'pubmed23n1163.xml.gz', 'pubmed23n1162.xml.gz', 'pubmed23n1161.xml.gz', 'pubmed23n1160.xml.gz', 'pubmed23n1159.xml.gz', 'pubmed23n1158.xml.gz', 'pubmed23n1157.xml.gz']
Predicted download size = 610MiB, Available space = 111GiB
SKIP: /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1166.xml.gz exists.
SKIP: /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1165.xml.gz exists.
SKIP: /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1164.xml.gz exists.
SKIP: /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1163.xml.gz exists.
SKIP: /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1162.xml.gz exists.
SKIP: /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1161.xml.gz exists.
SKIP: /

In [97]:
#%%bash
# this would probably be faster, but harder to maintain
#VERSION_ROOT=v1/data
#VERBOSE=1
#./gpubs/scripts/download_pubs.sh -n 5 -d ${VERSION_ROOT}/raw/pubs -v ${VERBOSE} 2> download.err

# Create CSVs from XMLs
functions in pubmed2

In [159]:
%%time
# takes about 3 minutes to do 10 files; or about 5 hours to do them all
csv_list = create_pubcsv_dataset(m)

Converting file /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1160.xml.gz
Number of all articles:29995
Number of all abstracts before pruning short articles = 27392
Number after pruning short articles = 18053
Number discarded for being too short: 9339
Number of pruned articles:18053
Wrote file:/home/krobasky/prompt/end2end/v1/data/csvpubs/pubmed23n1160.xml.gz.csv
Converting file /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1165.xml.gz
Number of all articles:29996
Number of all abstracts before pruning short articles = 25905
Number after pruning short articles = 16511
Number discarded for being too short: 9394
Number of pruned articles:16511
Wrote file:/home/krobasky/prompt/end2end/v1/data/csvpubs/pubmed23n1165.xml.gz.csv
Converting file /home/krobasky/prompt/end2end/v1/data/raw/pubs/pubmed23n1166.xml.gz
Number of all articles:10710
Number of all abstracts before pruning short articles = 9250
Number after pruning short articles = 5558
Number discarded for being too 

# Create new CSVs that include GENES column
use the search.awk script

Fix the awk script to swallow the entire csv but only match on the title, abstract columns; then output the whole thing.

In [160]:
def create_gene_files(m: ReferenceData):
    """ Calls the search.awk script in gpubs/scripts """
    
    filtered_terms_file = os.path.join(m.search_path(), m.filtered_terms_filename)
    csv_inpath = m.pub_outpath()
    csv_outpath = m.genes_outpath()
    verbose = m.verbose
    
    import glob
    import subprocess
    awk_script = "gpubs/scripts/search.awk"
    for file_name_path in glob.glob(os.path.join(csv_inpath,"pubmed*.xml.gz.csv")):
        file_name = os.path.basename(file_name_path)
        input_csv_file = os.path.join(csv_inpath, file_name)
        output_csv_file = os.path.join(csv_outpath, file_name)
        msg2(verbose, f"Creating {output_csv_file}")
        error_file = os.path.join(csv_outpath, f"{file_name}.err")
        command = [awk_script, filtered_terms_file, input_csv_file]
        with open(output_csv_file, "w") as output, open(error_file, "w") as error:
            subprocess.run(command, stdout=output, stderr=error)


In [161]:
m.genes_outpath()

'/home/krobasky/prompt/end2end/v1/data/csvpubs/genes/'

In [162]:
%%time
# this takes about 40s for 10 files, which is much slower than just running the awk script
# also, we filter out about 42% of the abstracts, most of which are 2022
# xxx ALSO - mask, flower, killer... these are all still in there, so the stop words aren't working, fix that.
create_gene_files(m)

Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1162.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1161.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1158.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1165.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1160.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1157.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1164.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1166.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1159.xml.gz.csv
Creating /home/krobasky/prompt/end2end/v1/data/csvpubs/genes/pubmed23n1163.xml.gz.csv
CPU times: user 39.9 ms, sys: 214 ms, total: 254 ms
Wall time: 22.5 s


In [166]:
!awk -F'\t' '$10 != ""{print $10}' ./v1/data/csvpubs/genes/*.xml.gz.csv|wc -l
! wc -l ./v1/data/csvpubs/genes/*.csv

91359
    15947 ./v1/data/csvpubs/genes/pubmed23n1157.xml.gz.csv
    17852 ./v1/data/csvpubs/genes/pubmed23n1158.xml.gz.csv
    16430 ./v1/data/csvpubs/genes/pubmed23n1159.xml.gz.csv
    18152 ./v1/data/csvpubs/genes/pubmed23n1160.xml.gz.csv
    15946 ./v1/data/csvpubs/genes/pubmed23n1161.xml.gz.csv
    17901 ./v1/data/csvpubs/genes/pubmed23n1162.xml.gz.csv
    13904 ./v1/data/csvpubs/genes/pubmed23n1163.xml.gz.csv
    17326 ./v1/data/csvpubs/genes/pubmed23n1164.xml.gz.csv
    16528 ./v1/data/csvpubs/genes/pubmed23n1165.xml.gz.csv
     5558 ./v1/data/csvpubs/genes/pubmed23n1166.xml.gz.csv
   155544 total


In [310]:
#%%bash
# This is SO much faster, but not as sustainable.
#./gpubs/scripts/search.awk \
#  ./v4/data/search_terms/filtered_terms.txt \
#  ./v4/data/csvpubs/pubmed23n1166.xml.gz.csv \
#> ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv 2> ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv.err
#
# field 10 has the genes
##bhead -1 ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv

35616278	Recent advances in anti-multidrug resistance for nano-drug delivery system.	Chemotherapy for tumors occasionally results in drug resistance, which is the major reason for the treatment failure. Higher drug doses could improve the therapeutic effect, but higher toxicity limits the further treatment. For overcoming drug resistance, functional nano-drug delivery system (NDDS) has been explored to sensitize the anticancer drugs and decrease its side effects, which are applied in combating multidrug resistance (MDR) via a variety of mechanisms including bypassing drug efflux, controlling drug release, and disturbing metabolism. This review starts with a brief report on the major MDR causes. Furthermore, we searched the papers from NDDS and introduced the recent advances in sensitizing the chemotherapeutic drugs against MDR tumors. Finally, we concluded that the NDDS was based on several mechanisms, and we looked forward to the future in this field.	Drug delivery	2022	Drug delivery	

In [167]:
%%bash
cat ./v1/data/csvpubs/genes/pubmed23n1166.xml.gz.csv.err
awk -F'\t' '$10 != "" {print $10}' ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv|head -120|tail -40


LD,MS,Rab,GO
CIs,Cox,CI,HR,HRs
MRSA,ERK,MEK
Ga,NPs,LMs,CO,nm
LOTUS
cage
tag,PS,MBP,PLD
digit,tip
nail
FST
MMR
ASD,ID
LRRK2
TLR4,MSU,MyD88,TLR2
HCC
CAFs,T,killer
MAT,PA
GRS,PA,CR
MRS,LCS
PH
lab,CH
OT
T
MRI,CT
MoS
flower,SRC
CS,GS
grasp
PhA
polymerase
spp,flower
MINT
pH
rain
P450,MES,bis
mask
Pb,Cr,P,pH,sand
CD68
HS
T1,T2


In [314]:
%%bash
cat ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv.err
awk -F'\t' '$10 != "" {print $10}' ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv|head -120|tail -40


LD,MS,Rab,GO
CIs,Cox,CI,HR,HRs
MRSA,ERK,MEK
Ga,NPs,LMs,CO,nm
LOTUS
cage
tag,PS,MBP,PLD
digit,tip
nail
FST
MMR
ASD,ID
LRRK2
TLR4,MSU,MyD88,TLR2
HCC
CAFs,T,killer
MAT,PA
GRS,PA,CR
MRS,LCS
PH
lab,CH
OT
T
MRI,CT
MoS
flower,SRC
CS,GS
grasp
PhA
polymerase
spp,flower
MINT
pH
rain
P450,MES,bis
mask
Pb,Cr,P,pH,sand
CD68
HS
T1,T2


In [306]:
%%bash
awk -F'\t' '$10 != "" {print $10}' ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv|head -40
#awk -F'\t' '$10 != "" {print $2"\n"$3"\n"$10,"\n"}' ./v4/data/csvpubs/genes/pubmed23n1166.xml.gz.csv|head -40

TEM,PI
CXCR4
AKT,BPA
SBS
A7,A7
PA,T,PI
CG
AFAP1-AS1,GRL
CSD,PAAf,FAA
Li,Li,SCL
ACTRT1,SPACA1,ACTL7A,ACTRT1,ACTRT2,ARP,SPATA46,PARP11,PT,envelope,ACTL9,KO
fate
Pinch2,Rho,Cdc42,IPP,hub,Pinch1,Ilk,Pinch2,RhoA
PSM,OC
Coma,DBS,GCS
clock,NAMPT,CLOCK,BMAL1,fat
E1,fat,GIP,E1,DO,fat
MALAT1,HDAC4,MALAT1,polymerase
CG,EG
NP
II
iNOS,eNOS,sGC
ES,HPO
IL-15,IL-15
ANOVA
MS,MT,RT
BRAF
FXR,YAP1,HCC,FXR,Yap1,YAP1
CO
POC,POC,Delta
km
EpCAM
OCT
AMPs,C-C,Fish,CCL28
H,Gb5
LHb
PHEs
PPCS,trio,PPCS
BPA,post
osf


# Wrap the pipeline

The pipeline should be all callable from python so you can use the model. 

wrap those bash scripts into python api's

# DONE: Package code

make a setup.py, conda.yml, etc.

put in github under krobasky: repo "gpubs"

In [41]:
%%bash

VERSION_ROOT="v1/data"
VERBOSE=2

./gpubs/scripts/create_search_terms_file.sh --root-dir $VERSION_ROOT \
   --ref-dir "reference" \
   --dbx-dir "dbxrefs" \
   --dbx "AllianceGenome.txt Ensembl.txt HGNC.txt IMGT_GENE-DB.txt" \
   --search-dir "search_terms" \
   --search-file "search_terms.txt" \
   --gene-symbols "gene_symbols.txt" \
   --gene-synonyms "gene_synonyms.txt" \
   --verbose $VERBOSE

Created v1/data/search_terms/search_terms.txt.
Created v1/data/search_terms/search_terms.txt.unsorted - can be removed.
Number of lines in v1/data/search_terms/search_terms.txt: 338143
